# Атака RSA Blinding
## Введение
RSA (названа в честь своих создателей Ronald Linn Rivest, Adi Shamir and Leonard Adleman) это пример асимметричной криптосистемы, которая может быть использована для безопасной передачи данных и создания подписей. рассмотрим базовые принципы работы криптосистемы.

RSA использует мультипликативную группу по модулю $N=pq$, где $p$ и $q$ - простые числа. Степень мультипликативной группы (по-сути, её мощность) может быть вычислена при помощи функции Эйлера  для составного числа из двух простых: $\varphi(N)=(p-1)(q-1)$. Функция считает количество натуральных чисел меньше $N$, которые не кратны $p$ или $q$. Поскольку все такие числа взаимно просты с $N$, они состоят в мультипликативной группу. Как мы знаем, если возвести любой элемент конечной мультипликативной группы в степень этой группы, то получим нейтральный элемент (единицу): $a^{\varphi(N)}=1, a \in Z^{*}_{N}$. Поэтому в RSA используют два числа $e$ (открытая экспонента) и $d$ (закрытая экспонента), такие что $ed=1 mod N$. Пара чисел $(e,N)$ используется как открытый ключ, а $(d,N)$ как закрытый. Вычисление $d$ из открытого ключа является сверхполиномиальной задачей (NP), если не были сгенерированы слабые $N$, $d$ или $e$. Одним из способов решения является факторизация $N$ в произведение $p$ и $q$.

Пусть дан открытый текст (число) $M, M < N$, открытый ключ $(e,N)$ и закрытый ключ $(d,N)$,  шифрование и расшифрование осуществляются следующим образом:

Шифрование
$C=M^{e}\space mod\space N$

Расшифрование

$M=C^{d} \space mod\space N$

Проверка корректности:

$C^{d}\space mod\space N=M^{ed}\space mod\space N= M^{ed\space mod\space \varphi(N)}\space mod\space N=M^{1}\space mod\space N= M\space mod\space N$

## Preparation
Попробуем немного поработать с RSA. Если ещё не установили, установите Pycryptodome. На Linux и Windows должна сработать следующая команда (Предварительно надо установить python 3 и pip, но я надеюсь, что вы справились с этим самостоятельно):

In [1]:
# !python3 -m pip install pycryptodome
# !conda install -c conda-forge pycryptodome

После установки надо перезапустить ядро jupyter (круговая стрелка рядом с "Run"). Если возникнут проблемы, загляните в документацию: [Pycryptodome installation](https://pycryptodome.readthedocs.io/en/latest/src/installation.html).

## Примитивная реализация RSA
Давайте сделаем простейшую версию RSA. Будем использовать открытую экспоненту $e=65537$. Обычно используют эту константу, потому что она переаолняет модуль даже при открытом тексте $M=2$ и у нее удобное двоичное представление $65537_{10}=10000000000000001_{2}$, которое позволяет эффективно возводить число в степень, используя алгоритм "Square and multiply".

Сначала сгенерируем $p$ и $q$. Функиция getStrongPrime дает возможность выбрать количество бит в генерируемом простом числе и проверяет, что $НОД(p-1,e)=1$

In [2]:
try:
    from Crypto.Util.number import getStrongPrime, inverse,bytes_to_long, long_to_bytes
except ImportError:
    print ("Pycryptodome not installed")

In [3]:
e=65537
p=getStrongPrime(1024,e=e)
q=getStrongPrime(1024,e=e)

In [4]:
N=p*q
phi=(p-1)*(q-1)
d=inverse(e,phi)
public_key=(e,N)
private_key=(d,N)

Мы успешно сгенерировали ключи, теперь давайте зашифруем сообщение, расшифруем закрытый текст и проверим, что получили то же самое

In [5]:
M=bytes_to_long(b'Hello, RSA!')
C=pow(M,e,N)
print ('C:',hex(C))
M1=pow(C,d,N)
assert M1==M
print ('M1:',long_to_bytes(M1))

C: 0xfc0a58d63d3af779a5838695689d426ec1eb4e6540fdcfcb2440d502c1fb6ff1577f6ab3c430ed3b73db50c20fef4a133732190f8984250f75d5bd54c44ed8e33f6dbe7d425b56c11925cb5ba5e59a46c869fc18d69f9c7f6cfedf599a966dfbce5cdaeae4f7c637d402a98f643350adf7157f95153f4da67f50c1a28e89a20cd6460c7bab7624a7bd0c08001eaa1ab25055c86d86f821d813a759173adcc6ca60a91b294b6d62b80c1190da8e928275d5ed243d6ea814666b2ba8f4e0b1f763f7141187a907585163bb5227a156cd77cb9fabf9cfd6613b6eb02e7ce5bdeffb6bc0a12dc989e9dc440a0eb08e1b33e817ec36ccaec894fd5662f4e523f934
M1: b'Hello, RSA!'


Создание подписи - обратная операция к шифрованию. 
$$Sign(M)\equiv Dec(M),\space Check(S) \equiv Enc(S)$$
Таким образом любой, владеющий окрытым ключом, может проверить правильность подписи, а создать её может только сторона, у которой есть закрытый ключ.
Поздравляю, теперь вы знаете, как шифровать и создавать подаписи при помощи RSA. Дальше рассмотрим одно из его интересных свойств.

## RSA Blinding
RSA - это гомоморфное шифрование по отношению к операции умножения.
Отношение является гомоморфизмом групп, если оно операцию одной группы переводит в опрацию другой группы ,операция и отношение элементов сохраняются. Если ничего не понятно, не беспокойтесь, я в первый раз, когда услышал, тоже ничего не понял. Что это значит на практике: пусть у вас есть два элемента группы $G_1$ $(x,y)$ и вы применяете к ним гомоморфное отображение, они будут также связаны в новой группе $G_2$ (для RSA $G_1= G_2$): 
$$\varphi(x\cdot y)=\varphi(x)\times\varphi(y)$$
Для шифрования RSA: $$Enc(M_1 \cdot M_2)=Enc(M_1)\times Enc(M_2)=$$
То же самое верно и для расшифрования:
$$Dec(C_1 \times C_2)=Dec(C_1) \cdot Dec(C_2)$$
Протестируем это свойство в python

In [6]:
class BasicRSA:
    def __init__(self, e,p,q):
        self.e=e
        self.p=p
        self.q=q
        self.N=p*q
        self.d=inverse(e,(p-1)*(q-1))
    
    def encryptNumber(self, m):
        return pow(m,self.e,self.N)
    
    def decryptNumber(self, c):
        return pow(c,self.d,self.N)

    
brsa=BasicRSA(e,p,q) #we created these parameters earlier
m1=2
m2=3
m3=m1*m2
c1=brsa.encryptNumber(m1)
c2=brsa.encryptNumber(m2)
print ('c1:',c1)
print ('c2:',c2)
c3=(c1*c2)%brsa.N
print('c3:',c3)
m3_dec=brsa.decryptNumber(c3)
print ('m3: %d, m3_dec: %d'%(m3,m3_dec))
assert m3_dec==m3

c1: 6673929034955853120153288332055348981140940082813052494222854023759320244259508162129462885544624798772669512913645532566715842218160186862151187781643632695180938702162743524018847715874147517790706962239080381768024757371926227237442829393779466139911248816408632627239100194087997735152774907470875773867735465183800006946086011360159278960590868165128556052869830634241177902223746408928336667742113973804628038343746908230101002199571444420498786826675939883993269461250388094953360738523936641368926324459245973370065339979521595116537222814964405287799047243003787262097606335624250914601998737390015725825471
c2: 165685391712889409589862269705472649152805785231784952569864230976706713212809606085376277072274128633887424323868533302905666086806572582209392943362587359381447587847857279370611702585996424488547188833870866704255257851277991475753200966877604951674231286354417483528403385677977827682335610277829771169595398393013548722623966660766878622540836239393949592695823212198444

## Атакуем сервер
Теперь попробуйте применить эти знания к уязвимому серверу. Вы можете приконнетиться, используя ```nc cryptotraining.zone 1337``` или при помощи питоновских сокетов.

In [7]:
import socket
import re
class VulnServerClient:
    def __init__(self,show=True):
        """Ининциализация, подключаемся к серверу"""
        self.s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        self.s.connect(('cryptotraining.zone',1337))
        if show:
            print (self.recv_until().decode())
    def recv_until(self,symb=b'\n>'):
        """Получение сообщения с сервера, по умолчанию до приглашения к вводу команды"""
        data=b''
        while True:
            
            data+=self.s.recv(1)
            if data[-len(symb):]==symb:
                break
        return data
    def get_public_key(self,show=True):
        """Получение открытого ключа с сервера"""
        self.s.sendall('public\n'.encode())
        response=self.recv_until().decode()
        if show:
            print (response)
        e=int(re.search('(?<=e: )\d+',response).group(0))
        N=int(re.search('(?<=N: )\d+',response).group(0))
        self.num_len=len(long_to_bytes(N))
        return (e,N)
    
    def signBytes(self,m,show=True):
        """Получение подписи для выбранного сообщения в байтах с сервера"""
        try:
            num_len=self.num_len
        except KeyError:
            print ('You need to get the public key from the server first')
            return
        if len(m)>num_len:
            print ("The message is too long")
            return
        if len(m)<num_len:
            m=bytes((num_len-len(m))*[0x0]) +m
        hex_m=m.hex().encode()
        self.s.sendall(b'sign '+hex_m+b'\n')
        response=self.recv_until().decode()
        if show:
            print (response)
        if response.find('flag')!=-1:
            print('You tried to submit \'flag\'')
            return None
        signature_hex=re.search('(?<=Signature: )[0-9a-f]+',response).group(0)
        signature_bytes=bytes.fromhex(signature_hex)
        return bytes_to_long(signature_bytes)
    
    
    def signNumber(self,m,show=True):
        """Получение подписи с сервера для выбранного сообщения в числовом представлении"""
        try:
            num_len=self.num_len
        except KeyError:
            print ('You need to get the public key from the server first')
            return
        return self.signBytes(long_to_bytes(m,num_len),show)
        
    def checkSignatureNumber(self,c,show=True):
        """Проверка сигнатуры (на сервере) для подписи в числовом представлении"""
        try:
            num_len=self.num_len
        except KeyError:
            print ('You need to get the public key from the server first')
            return
        signature_bytes=long_to_bytes(c,num_len)
        self.checkSignatureBytes(signature_bytes,show)
    
    def checkSignatureBytes(self,c,show=True):
        """Проверка сигнатуры (на сервере) для подписи в байтовом представлении"""
        try:
            num_len=self.num_len
        except KeyError:
            print ('You need to get the public key from the server first')
            return
        if len(c)>num_len:
            print ("The message is too long")
            return
        
        hex_c=c.hex().encode()
        self.s.sendall(b'flag '+hex_c+b'\n',)
        response=self.recv_until(b'\n').decode()
        
        if show:
            print (response)
        
        if response.find('Wrong')!=-1:
            print('Wrong signature')
            x=self.recv_until()
            if show:
                print (x)
            return
        flag=re.search('CRYPTOTRAINING\{.*\}',response).group(0)
        print ('FLAG: ',flag)
        
    def __del__(self):
        self.s.close()

In [8]:
vs=VulnServerClient()
(e,N)=vs.get_public_key()

Welcome to RSA blinding task
Available commands:
help - print this help
public - show public key
sign <hex(data)> - sign data
flag <hex(signature(b'flag'))> - print flag 
quit - quit
>
e: 65537
N: 20159717663186764200842482638329142432479376755681286432561400011207751568770239378735042390550988864636478212097889382541806378632813451522011734778394352464750695430236459156439656932108536936107092785759187120915559173321302027525229018106368725032056109022369913503577023942696069608771010384365856481001383579432844112231215767630328627015097422540087789462404508697086321213990868031273219614897901436844999442259387453021270642395531884848697650933478124254071912232445708062597679170291021925633789812405697682134528381868778865376836541179591638312152472136313757252384761293684336082840137773984575947459061
>


Вы можете подписывать сообщения при помощи методов signNumber (подписать число) и signBytes (подписать сообщение из байтов)

Проверять подпись можете при помощи методов checkSignatureNumber и checkSignatureBytes.

Ваша цель - получить правильную подпись для сообщения 'flag'.

Помните, что RSA - это гомоморфизм и решите задание.

Удачи!

In [9]:
int_flag = bytes_to_long(b'flag')

In [10]:
int_flag

1718378855

In [14]:
for_sign=[5,int(int_flag/5)]

In [15]:
import numpy as np

In [16]:
l=[vs.signNumber(n) for n in for_sign ]

Signature: 3db3b9da37432567eabd6408073feae549e9831e4f3ff63aa9314f2a2df55954d7287f2b735b67bfafd89121f4b90aa9cb59fb8a84e0432e22b9c87a116c4e1c224923ab44c460ee98176e784eede7c6df61a7f3adac690d24b9df8d0273cdc54ae8bb5ef27cb33f38a592ffccbb74031952d26fe6ba934204e0cabb09aca86495fda391c6a2ab2a32e7e935220087baf224c05f35bcf02b39af9dce3bb6a5fe7d871c1f91267e2be0a5e39aec8184107c9d09407ef63bf42585abd49c12fed532749782e475ddc99da8fc86f52507a5d0abc6bfdcce43e3b4fe83b0407191bc196f2121b79f67464640597ecd8f856cc210be288a4bc8a40ba179272eac1e9c
>
Signature: 3dd9ecff216fd8d3f72957971a8e681a9c05134e34da675ed9a4397d681558033cb3c0d5350035008161cc3a34948cdd8351c4ec233fddfe29fa6517192f1048757f64d9d891bd4c8c2403ad423650a06824bf8e02ee761317bd9f6fb1135fa2b1ea7d9c1eef42e704c625caaaa652cbc99fc765c861790a65b78aba0456457b7eba97cff511fc3156e905b0f561007963753460bbb2ed02ebea9e182a181455b08626558b06ade0594fb006779d9eee3dd9efa96db35dcc4804571903453b8843cdb9f108a7e9686cff3d47d3a537745f17bad1ec06fc308ce51d79d1027b4d79ae1aae72e8ae7

In [17]:
l

[7789162789749622940582660203446907339704052033964996544507275348409627662417224482809969326593440791850888296235214696238413963388219581377486577206498622720603889359784722195830822473924119731869728173599490469130323090483878131135201142883889358676347124578027537843477673445808581213162046030560672655370837317346028962261230322788296559613715314707640020065925622907680318672474207030713180370996449053713398240459347198643229984240684834809369089179880967369379741082125542239926264136302760061995523672928779339299444899322784325159716587928299146357109152969757267037781460728366007779985084928089535466446492,
 78079998043309536282462623302223897896852746030482687516725269855058369270002495455385176787354002811018991172844753838840410800262608896219699721632192922396857527431446534850233628882295828650262554892752820309745754943613179431549972358175221505264895222315855778511629297525223299634209554477446770064293726501595292384497514383956369358895203322979784002070469992052426846609

In [18]:
result = np.prod(l)%N

In [19]:
result

19303065881999525816204078786563336411861759575841294492506193209545168654560598894859368564428794656908589685051103822092716073686725032450919116902231529277269791196874260437187429626674591185993481599160378107435534788968522040264991522918096402547869837198558091471378883744264064414452875584436475801860022266239772906674578895824112180094641100916140571431826344215125915717734889889399910140472020540115418687541770929086654095933653235881547636032160981122844597189282094757478623830956567790495865710667243205139541701961628132149783462885090079596829740273219439077835085863619313219408780905770744750966264

In [20]:
vs.checkSignatureNumber(result)

Congratulations! Here is your flag: CRYPTOTRAINING{n0t_s0_bl1nd_4ft3r_4ll}

FLAG:  CRYPTOTRAINING{n0t_s0_bl1nd_4ft3r_4ll}


In [ ]:
# bytes_to_long, long_to_bytes